# Spaces

Spaces are handled in `ebcc` according to the following nomenclature:

                ─┬─ ┌──────────┐
                 │  │  frozen  │
                 │  ├──────────┤ ─┬─
         virtual │  │  active  │  │
                 │  ├──────────┤  │ correlated
                 │  │ inactive │  │
                ─┼─ ├══════════┤ ─┼─
                 │  │ inactive │  │
                 │  ├──────────┤  │ correlated
        occupied │  │  active  │  │
                 │  ├──────────┤ ─┴─
                 │  │  frozen  │
                ─┴─ └──────────┘

A plain coupled cluster calculation will operate within the correlated space. The simplest use of this system is frozen core calculations.

First, we again find system and mean-field using PySCF, and initialise an appropriate logger.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="H 0 0 0; F 0 0 1.1", basis="cc-pvdz", verbose=0)
mf = scf.RHF(mol).run()

In [12]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

The `Space` class is constructed by providing boolean arrays indicating whether an orbital at that index is occupied, frozen, or active. In this example we freeze the two lowest energy MOs.

In [4]:
from ebcc import Space

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True

space = Space(occupied, frozen, active)

print("Space:", space)

Space: (5o, 14v) [(2o, 0v) frozen]


The `active` space is used for methods that differentiate between the correlated orbitals in order to treat a subset of orbitals at a higher level of theory. The `frozen` and `active` arrays must be disjoint.

In [11]:
occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:1] = True
active[np.sum(mf.mo_occ > 0) - 1] = True
space = Space(occupied, frozen, active)
print("Space:", space)

try:
    active = frozen
    space = Space(occupied, frozen, active)
except ValueError as e:
    print("Error:", e)

Space: (5o, 14v) [(1o, 0v) frozen, (1o, 0v) active]
Error: Frozen and active orbitals must be mutually exclusive.


The space can be used in a calculation to perform the frozen-core CC calculation. All methods should be compatible with this procedure, as the code generation is agnostic to the definition of the space.

In [14]:
from ebcc import REBCC

occupied = mf.mo_occ > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True
space = Space(occupied, frozen, active)

cc2 = REBCC(mf, space=space, log=log)
cc2.kernel()

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


RCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: (5o, 14v) [(2o, 0v) frozen]

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.1547617180    -100.1421591584
   1    -0.1561846863    -100.1435821267     1.423e-03     8.569e-03
   2    -0.1578343707    -100.1452318110     1.650e-03     4.811e-03
   3    -0.1583435249    -100.1457409653     5.092e-04     2.697e-03
   4    -0.1584821749    -100.1458796152     1.386e-04     8.051e-04
   5    -0.1584614174    -100.1458588577     2.076e-05     2.810e-04
   6    -0.1584609032 

-0.15846102874067264